In [1]:
from datasets import load_dataset, load_metric, load_from_disk, Audio
from datasets import ClassLabel, DatasetDict, Dataset
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
import IPython.display as ipd
import numpy as np
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer
from evaluate import load
import os
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import argparse
import json
import logging
import os
import subprocess
import tarfile
import urllib.request

Dataset

In [19]:
#gale_speech = load_dataset('csv', data_files=['train13.csv', 'train14.csv'])
gale_speech = load_dataset('csv', data_files=['short_train_2013.csv', 'short_train_2014.csv'])
gale_speech = gale_speech.rename_column('transcript', 'text')
gale_speech

Using custom data configuration default-a53efedd5fb11b01
Found cached dataset csv (/home/ujan/.cache/huggingface/datasets/csv/default-a53efedd5fb11b01/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio_path', 'text', 'duration'],
        num_rows: 107016
    })
})

In [20]:
dataset = gale_speech.cast_column("audio_path", Audio())
dataset = dataset.rename_column('audio_path', 'audio')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))  # downsample to 16k
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 107016
    })
})

In [21]:
#dataset = dataset.filter(lambda example: example['audio'] is not None)
dataset = dataset.filter(lambda example: example['text'] is not None)
dataset

Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/csv/default-a53efedd5fb11b01/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e66b721c73199440.arrow


DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 105182
    })
})

Split into train test

In [22]:
dataset = dataset['train'].train_test_split(test_size=0.1)  # 178, 20
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 94663
    })
    test: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 10519
    })
})

In [24]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"].remove_columns(["duration", "audio"]))

,text
0,何先生负责什么
1,如果是两个罪犯他能这样干吗
2,你认为历史上最恐怖的恐怖片是哪一部呢一会儿我们来看一看网友的评选
3,分析人士指出尽管没有机会和布什在联大举行辩论
4,还有近十万的群众电话预约了参观
5,那我们应该如何来看待苏贞昌这种甘愿为民进党利益而
6,听取党员对二零零八年台湾地区领导人选举的意见建议
7,二零零四年建成在北京上海广州三城市进行了联网试运行
8,否则单凭我们老百姓的根本无法辨认哪些是假药哪些是放心药
9,呃谢谢


Get vocab

In [25]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'  ######## chinese?

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

dataset = dataset.map(remove_special_characters)

  0%|          | 0/94663 [00:00<?, ?ex/s]

  0%|          | 0/10519 [00:00<?, ?ex/s]

In [26]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

We use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class.

In [29]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'醉': 0,
 '猛': 1,
 '朗': 2,
 '皇': 3,
 '躲': 4,
 '疚': 5,
 '见': 6,
 '來': 7,
 '狡': 8,
 '捣': 9,
 '凌': 10,
 '不': 11,
 '槽': 12,
 '鹫': 13,
 '笤': 14,
 '涂': 15,
 '镑': 16,
 '系': 17,
 '勺': 18,
 '鞍': 19,
 '假': 20,
 '郭': 21,
 '微': 22,
 '位': 23,
 '醛': 24,
 '饕': 25,
 '窟': 26,
 '温': 27,
 '赏': 28,
 '哦': 29,
 '萎': 30,
 '史': 31,
 '矛': 32,
 '拍': 33,
 '哑': 34,
 '歇': 35,
 '货': 36,
 '异': 37,
 '芯': 38,
 '凸': 39,
 '得': 40,
 '穗': 41,
 '甸': 42,
 '芒': 43,
 '千': 44,
 '辉': 45,
 '屋': 46,
 '齐': 47,
 '踟': 48,
 '陇': 49,
 '韩': 50,
 '订': 51,
 '甜': 52,
 '耀': 53,
 '仲': 54,
 '娩': 55,
 '岑': 56,
 '宝': 57,
 '草': 58,
 '酮': 59,
 '诩': 60,
 '伎': 61,
 '决': 62,
 '搂': 63,
 '末': 64,
 '殖': 65,
 '慨': 66,
 '除': 67,
 '柘': 68,
 '雾': 69,
 '送': 70,
 '扒': 71,
 '绪': 72,
 '恩': 73,
 '呢': 74,
 '荟': 75,
 '铭': 76,
 '汕': 77,
 '铢': 78,
 '稿': 79,
 '讨': 80,
 '肃': 81,
 '洪': 82,
 '姓': 83,
 '汀': 84,
 '宛': 85,
 '挠': 86,
 '腰': 87,
 '琉': 88,
 '您': 89,
 '窍': 90,
 '黯': 91,
 '堕': 92,
 '帼': 93,
 '劫': 94,
 '題': 95,
 '缝': 96,
 '氢': 97,
 '肮': 98,
 '慈': 99,
 '遁': 100,

Wav2Vec2 feature extractor

In [30]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

4110


To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object

In [31]:
with open('gale_speech_vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [16]:
tokenizer = Wav2Vec2CTCTokenizer("./gale_speech_vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [17]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [18]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [35]:
dataset['train'][0]

{'audio': {'path': 'gale_p2_cmn_bn_speech_LDC2013S08/gale_p2_cmn_bn_speech/slices2/PHOENIX_PHNXWRLD_CMN_20061006_112400_33.wav',
  'array': array([-0.0016982 , -0.00150405, -0.00054022, ..., -0.00731185,
         -0.00530905,  0.        ], dtype=float32),
  'sampling_rate': 16000},
 'text': '所以就是说从我个人角度来看也是一样的',
 'duration': 3.468}

In [36]:
rand_int = random.randint(0, len(dataset['train'])-1)

print(dataset['train'][rand_int]["text"])
ipd.Audio(data=dataset['train'][rand_int]["audio"]["array"], autoplay=True, rate=16000)

法庭上原告律师出示了一份调查笔录调查对象正是将王阿姨介绍给程家的保姆介绍所


In [37]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [38]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=4)

#2:   0%|          | 0/23666 [00:00<?, ?ex/s]

#1:   0%|          | 0/23666 [00:00<?, ?ex/s]

#0:   0%|          | 0/23666 [00:00<?, ?ex/s]

#3:   0%|          | 0/23665 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your l

#0:   0%|          | 0/2630 [00:00<?, ?ex/s]

#1:   0%|          | 0/2630 [00:00<?, ?ex/s]

#3:   0%|          | 0/2629 [00:00<?, ?ex/s]

#2:   0%|          | 0/2630 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your l

In [40]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 94663
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 10519
    })
})

In [41]:
dataset.save_to_disk('/home/ujan/Downloads/Gale/gale_speech_vectorized')

Saving the dataset (0/52 shards):   0%|          | 0/94663 [00:00<?, ? examples/s]

Saving the dataset (0/6 shards):   0%|          | 0/10519 [00:00<?, ? examples/s]

In [2]:
dataset = load_from_disk('/home/ujan/Downloads/Gale/gale_speech_vectorized')

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 94663
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 10519
    })
})

In [14]:
demo = DatasetDict()
demo['train'] = dataset['train'].select([i for i in range(10000)])
demo['val'] = dataset['test'].select([i for i in range(1000)])

demo

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 10000
    })
    val: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 1000
    })
})

Training & Evaluation

In [19]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [20]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [21]:
#wer = load("wer")
cer_metric = load("/home/ujan/Downloads/evaluate/metrics/cer/cer.py")

In [22]:
model = Wav2Vec2ForCTC.from_pretrained(
    "/home/ujan/Downloads/aishell1_finetune/mandarin-wav2vec2",
    attention_dropout=0.1,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /home/ujan/Downloads/aishell1_finetune/mandarin-wav2vec2 and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([4233, 768]) in the checkpoint and torch.Size([4110, 768]) in the model instantiated
- lm_head.bias: found shape torch.Size([4233]) in the checkpoint and torch.Size([4110]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the paper does not need to be fine-tuned anymore 

In [23]:
model.freeze_feature_encoder()

In [24]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    #wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [27]:
training_args = TrainingArguments(
    output_dir='/home/ujan/Notebooks/aishell_ctc',
    group_by_length=True,
    #per_device_train_batch_size=32,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_accumulation_steps=4,
    num_train_epochs=20,
    fp16=True,
    gradient_checkpointing=True, 
    save_steps=500, #
    eval_steps=500, #
    logging_steps=500, #
    learning_rate=0.0002,
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=2,
    dataloader_num_workers=0,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=demo["train"],
    eval_dataset=demo["val"], # validation
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [29]:
trainer.train()

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 12500
  Number of trainable parameters = 93331854
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warni

Step,Training Loss,Validation Loss,Cer
500,27.157200,8.821177,1.000000
1000,6.715900,9.496301,1.000000
1500,6.706400,8.648768,1.000000
2000,6.696000,9.164731,1.000000
2500,6.698800,9.415912,1.000000


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/aishell_ctc/checkpoint-500
Configuration saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/config.json
Model weights saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/aishell_ctc/checkpoint-500/preprocessor_config.json
/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/aishell_ctc/checkpoint-1000
Configuration saved in /home/ujan/Notebooks/aishell_

KeyboardInterrupt: 

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 109973
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 12220
    })
})

In [22]:
len(dataset['train'][23]['input_values'])

92529

In [ ]:
aishell = load_

Evaluation

In [15]:
device = torch.device("cuda")
model.to(device)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)

In [16]:
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
    return batch

results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

  0%|          | 0/7176 [00:00<?, ?ex/s]

In [17]:
print("Test CER: {:.3f}".format(cer.compute(predictions=results["pred_str"], references=results["text"])))

Test CER: 7.120


In [18]:
results_val = dataset["validation"].map(map_to_result, remove_columns=dataset["test"].column_names)

  0%|          | 0/14326 [00:00<?, ?ex/s]

In [24]:
print("Val CER: {:.3f}".format(cer.compute(predictions=results_val["pred_str"], references=results_val["text"])))

Val CER: 6.723
